<a href="https://colab.research.google.com/github/Yomunozs/Aprendizaje_Maquina/blob/main/Parciales/Parcial_1_TAM_Pr%C3%A1ctica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

2 (Valor 2.5 puntos) Genere una simulacion sobre Python de los regresores por m ´ axima verosimilitud y m ´ aximo a-posteriori, discu- ´
tidos en el punto 2.1, para ajustar la senal: ˜ tn = cos[xn/3] + cos[xn/4] + ηn, con xn ∈ [0, 24π], contaminada con ruido blanco
Gaussiano ηn (SNRdB = 2[dB]). Asuma mapeo ϕ(·) del tipo polinomial de orden Q y prior p(w) = N (w|0, σ2
w). Simule 500
datos para entrenar los modelos y 200 para predecir. Incluya normalizacion por MinMaxScaler() de sklearn despu ´ es de generar el ´
mapeo no lineal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import ShuffleSplit
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

In [ ]:
N = 700
x_n = np.linspace(0,24*np.pi,N)
t_n = np.cos(x_n/3)+np.cos(x_n/4)

#función cálculo varianza del ruido a partir del snr dB
def var_snr(x,snrdB): #x vector de datos (señal), snrdB SNR en dB
    Px = np.mean(x**2)#estimador potencia media de la señal
    return Px/(10**(snrdB/10))
snrdB = 2
var_r = var_snr(t_n,snrdB)


t_nr = t_n + np.sqrt(var_r)*np.random.randn(N)


x_n = x_n.reshape(-1,1)#filas = realizaciones-muestras
t_nr = t_nr.reshape(-1,1)

plt.figure(figsize = (5, 3))
plt.plot(x_n, t_n, label='$t_n^*$', c='r', linewidth=2)
plt.scatter(x_n, t_nr, c='b', label='$t_n$')
plt.legend()
plt.xlabel('x_n')
plt.ylabel('t_n')
plt.show()




In [ ]:
#mapeo no lineal
x_train, x_test, t_train, t_test = train_test_split(x_n, t_nr,
                                                    test_size=200/N, random_state=42)

print(x_train.shape,x_test.shape,t_train.shape,t_test.shape)

In [ ]:
#rs = ShuffleSplit(n_splits=1, random_state=0, test_size=0.2)
#for i, (x_train, x_test) in enumerate(rs.split(x_n)):
#   print(i)


In [ ]:
#generación representación polinomial, desde la libreria sklearn
Q = 12 #grado del polinomio
phiQ = PolynomialFeatures(degree=Q)
Phi_xn = phiQ.fit_transform(x_n)#representar datos desde polinomio

In [ ]:
#print(x_train.shape, x_test.shape)
#scaler = MinMaxScaler()
#x_train = scaler.fit_transform(x_train.reshape(-1,1))
#x_test = scaler.transform(x_test.reshape(-1,1))

#x_train.reshape(-1,1)
#x_test.reshape(-1,1)
print(x_train[:,0].shape, x_test.shape)

In [ ]:
#regresor
lambdaR = 1e-15#hiperparámetro de regularización
reg_mc = Ridge(alpha=lambdaR)

x_train = np.sort(x_train)
x_test = np.sort(x_test)

reg_mc.fit(Phi_xn[x_train],t_nr[x_train])

t_mc = reg_mc.predict(Phi_xn[x_test])

In [ ]:
plt.figure(figsize=(5,3))
plt.plot(x_n,t_n,c='r',label='$t^*$')
plt.scatter(x_n,t_nr,c='b',label='$t$')
plt.plot(x_test,t_mc,c='g',label='$t_{mcr}$')

plt.legend()
plt.xlabel('x')
plt.ylabel('t')
plt.show()

plt.figure(figsize=(5,3))
plt.stem(reg_mc.coef_[0])
plt.ylabel('pesos')
plt.show()

In [ ]:
#gráfica estimación con predictiva en ML:
var_ml = (np.linalg.norm(t_train-reg_mc.predict(Phi_xn[x_train]))**2)/len(t_nr[t_train])

plt.figure(figsize=(5,3))
plt.plot(x_n,t_n,c='r',label='$t$')
plt.scatter(x_n,t_nr,c='b',label='$t+\eta$')
plt.plot(x_test,t_mc,c='g',label='$t_{*}$')
plt.fill_between(x_test.ravel(), t_mc.ravel() - np.sqrt(var_ml)*np.ones(len(t_mc)),
                t_mc.ravel() + np.sqrt(var_ml)*np.ones(len(t_mc)), alpha=0.2)

plt.legend()
plt.xlabel('x')
plt.ylabel('t')
plt.show()

print(var_ml)

In [ ]:
#solución Maxima Verosimilitud

def my_ML(phi, t_n):
  w_ml = np.linalg.inv(phi.T @ phi) @ phi.T @ t_n

  s_ml = 1


  return w_ml, s_ml

w, s = my_ML(Phi_xn, t_n)
phiN  = phiQ.fit_transform(x_test)#representar datos desde polinomio
phiN = np.sort(phiN)
tn =  phiN @ w
plt.plot(x_test,tn)
plt.plot(x_n,t_n)

In [ ]:


#def my_map():
#return

#...